<a href="https://colab.research.google.com/github/DanielACocolete/Q-Learning/blob/main/Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O que é Q-Learning?
O Q-Learning é uma técnica de aprendizado por reforço que visa otimizar a função de valor (ou Q-value) com base no ambiente ou problema em questão. Aqui estão os principais pontos:

Aprendizado por Reforço (RL): Imagine treinar um animal de estimação, recompensando-o a cada resposta correta. O RL segue uma lógica semelhante, onde um agente (como um programa de computador ou um robô) aprende a executar tarefas eficazmente com base em recompensas. Diferentemente da aprendizagem supervisionada, o RL não possui respostas corretas predefinidas; os agentes aprendem com a experiência e tomam decisões com base nas ações que maximizam a recompensa1.
Q-Learning: O “Q” em Q-Learning representa a “qualidade” com que o modelo escolhe sua próxima ação, buscando melhorar essa qualidade ao longo do tempo. O algoritmo armazena valores em uma tabela chamada Tabela Q, que representa a função de valor. Simplificando, o Q-Learning busca aprender uma política que maximize a recompensa total. Ele é surpreendentemente útil para iniciar sua jornada no aprendizado por reforço1.
Diferença entre Algoritmo e Modelo:
Algoritmo de Aprendizado de Máquina: Um conjunto de regras e procedimentos matemáticos e estatísticos usado pelo modelo de aprendizado de máquina para identificar padrões e fazer previsões com base em dados.
Modelo de Aprendizado de Máquina: Um programa que toma decisões ou faz previsões com base em dados, aprendendo com exemplos anteriores. No caso do Q-Learning, o algoritmo é parte do modelo, e o modelo é a tabela Q que armazena os valores de ação1.
Q-Learning com Redes Neurais
Aqui está a parte interessante: podemos combinar o Q-Learning com redes neurais para criar o Q-Learning Profundo (ou Deep Q-Learning). Nesse caso, utilizamos redes neurais artificiais para estimar a função Q de maneira mais eficiente. As redes neurais profundas (DQNs) integram-se ao Q-Learning, permitindo lidar com problemas mais complexos. Como funciona?

In [1]:
!pip install gymnasium[classic_control]  # Instala o Gym (fork do OpenAI Gym)
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from collections import deque

# Definindo a rede neural DQN
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Memória de Replay
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def add(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return np.vstack(states), actions, rewards, np.vstack(next_states), dones

# Função de ação com epsilon-greedy
def act(state, epsilon):
    if random.random() < epsilon:
        return env.action_space.sample()
    else:
        state = torch.FloatTensor(state).unsqueeze(0)
        q_values = agent(state)
        return np.argmax(q_values.detach().numpy())

# Parâmetros
state_size = 4  # Tamanho do vetor de estado (CartPole)
action_size = 2  # Número de ações (esquerda ou direita)
batch_size = 32
gamma = 0.99  # Fator de desconto
epsilon = 1.0  # Taxa de exploração inicial
epsilon_min = 0.01
epsilon_decay = 0.995
learning_rate = 0.001
target_update = 10  # Frequência de atualização da rede alvo

# Configuração do ambiente
env = gym.make('CartPole-v1')
agent = DQN(state_size, action_size)
target_net = DQN(state_size, action_size)  # Rede alvo
target_net.load_state_dict(agent.state_dict())
optimizer = optim.Adam(agent.parameters(), lr=learning_rate)
memory = ReplayBuffer(10000)

# Treinamento
episodes = 300
for episode in range(episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0

    while not done:
        action = act(state, epsilon)
        next_state, reward, done, _, _ = env.step(action)
        memory.add(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

        # Atualizar a rede neural
        if len(memory.buffer) >= batch_size:
            states, actions, rewards, next_states, dones = memory.sample(batch_size)
            states = torch.FloatTensor(states)
            next_states = torch.FloatTensor(next_states)
            rewards = torch.FloatTensor(rewards)
            dones = torch.FloatTensor(dones)

            q_values = agent(states).gather(1, torch.LongTensor(actions).unsqueeze(1)).squeeze(1)
            next_q_values = target_net(next_states).max(1)[0]
            targets = rewards + gamma * next_q_values * (1 - dones)

            loss = (q_values - targets).pow(2).mean()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Atualizar a rede alvo
        if episode % target_update == 0:
            target_net.load_state_dict(agent.state_dict())

    # Reduzir epsilon (exploração)
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    print(f'Episódio {episode}, Total de Recompensas: {total_reward}')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 9.6 MB/s eta 0:00:00
Episódio 0, Total de Recompensas: 13.0
Episódio 1, Total de Recompensas: 27.0
Episódio 2, Total de Recompensas: 49.0
Episódio 3, Total de Recompensas: 15.0
Episódio 4, Total de Recompensas: 27.0
Episódio 5, Total de Recompensas: 15.0
Episódio 6, Total de Recompensas: 12.0
Episódio 7, Total de Recompensas: 19.0
Episódio 8, Total de Recompensas: 21.0
Episódio 9, Total de Recompensas: 12.0
Episódio 10, Total de Recompensas: 19.0
Episódio 11, Total de Recompensas: 20.0
Episódio 12, Total de Recompensas: 13.0
Episódio 13, Total de Recompensas: 44.0
Episódio 14, Total de Recompensas: 36.0
Episódio 15, Total de Recompensas: 12.0
Episódio 16, Total de Recompensas: 26.0
Episódio 17, Total de Recompensas: 23.0
Episódio 18, Total de Recompensas: 19.0
Episódio 19, Total de Recompensas: 11.0
Episódio 20, Total de Recompensas: 20.0
Episódio 21, Total de Recompensas: 44.0
Episódio 22, Total de Recompensas: 13.0
Episódio 2

KeyboardInterrupt: 